In [1]:
from glob import glob
from pympi import Elan
import os
import pandas as pd
import enchant

# Make csv with EAF annotations

In [66]:
eaf_raw_path='../data/tira-asr/eaf_raw.csv'
df = pd.read_csv(eaf_raw_path, keep_default_na=False)
df.head()


,start,end,transcription,eaf_source,wav_source
0,184240,189330,mə̀nà tʃàt̪á án âjt̪ɔ́ ŋìðə̀nɔ́ŋ ŋírɔ́t...,/Users/markjos/Library/CloudStorage/GoogleDriv...,/G:/Shared drives/Tira/Recordings/2022-07-19-T...
1,244720,245840,mna tʃat̪a,/Users/markjos/Library/CloudStorage/GoogleDriv...,/G:/Shared drives/Tira/Recordings/2022-07-19-T...
2,247990,249050,an ajt̪ɔ,/Users/markjos/Library/CloudStorage/GoogleDriv...,/G:/Shared drives/Tira/Recordings/2022-07-19-T...
3,257670,258760,an ajit̪ɔ ŋiðinɔŋ,/Users/markjos/Library/CloudStorage/GoogleDriv...,/G:/Shared drives/Tira/Recordings/2022-07-19-T...
4,260180,261110,ŋirɔt̪ɛ,/Users/markjos/Library/CloudStorage/GoogleDriv...,/G:/Shared drives/Tira/Recordings/2022-07-19-T...


# Normalize diacs and remove toneless entries

In [67]:
import unicodedata

In [68]:
df['unicode_norm'] = df['transcription'].apply(lambda s: unicodedata.normalize('NFKD', s))
df.head()

,start,end,transcription,eaf_source,wav_source,unicode_norm
0,184240,189330,mə̀nà tʃàt̪á án âjt̪ɔ́ ŋìðə̀nɔ́ŋ ŋírɔ́t...,/Users/markjos/Library/CloudStorage/GoogleDriv...,/G:/Shared drives/Tira/Recordings/2022-07-19-T...,mə̀nà tʃàt̪á án âjt̪ɔ́ ŋìðə̀nɔ́ŋ ŋírɔ́t...
1,244720,245840,mna tʃat̪a,/Users/markjos/Library/CloudStorage/GoogleDriv...,/G:/Shared drives/Tira/Recordings/2022-07-19-T...,mna tʃat̪a
2,247990,249050,an ajt̪ɔ,/Users/markjos/Library/CloudStorage/GoogleDriv...,/G:/Shared drives/Tira/Recordings/2022-07-19-T...,an ajt̪ɔ
3,257670,258760,an ajit̪ɔ ŋiðinɔŋ,/Users/markjos/Library/CloudStorage/GoogleDriv...,/G:/Shared drives/Tira/Recordings/2022-07-19-T...,an ajit̪ɔ ŋiðinɔŋ
4,260180,261110,ŋirɔt̪ɛ,/Users/markjos/Library/CloudStorage/GoogleDriv...,/G:/Shared drives/Tira/Recordings/2022-07-19-T...,ŋirɔt̪ɛ


In [69]:
tone_markers = {
    'grave': "\u0300",
    'macrn': "\u0304",
    'acute': "\u0301",
    'circm': "\u0302",
    'caron': "\u030C",
}
has_tone = lambda s: any(diac in s for diac in tone_markers.values())

print(len(df))
df_tone=df.copy()[df['unicode_norm'].apply(has_tone)]
len(df_tone)

28820


26697

# Remove punctuation

In [70]:
from string import punctuation

In [71]:
has_punct = lambda s: any(c in punctuation for c in s)
def punct_to_space(s):
    for c in punctuation:
        s=s.replace(c, ' ')
    return s
normalize_spaces = lambda s: ' '.join(s.split())
remove_punct = lambda s: normalize_spaces(punct_to_space(s))
has_punct('hello!?'), remove_punct('h?ell:;,.o?!&')

(True, 'h ell o')

In [72]:
df_tone.loc[df_tone['unicode_norm'].apply(has_punct), 'unicode_norm']

0        mə̀nà tʃàt̪á án âjt̪ɔ́ ŋìðə̀nɔ́ŋ ŋírɔ́t...
15       mnà tʃàt̪á án ájít̪ɔ́ ŋìðìnɔ́ŋ ŋ̀rɔ́t̪...
16       án âjt̪ɔ́ ŋíðə̀nɔ́ŋ ŋʊ̀rɔ́t̪ɛ̌, íŋgáŋâɽi...
18       án âjt̪ɔ́ ŋíðə̀nɔ́ŋ ŋʊ̀rɔ́t̪ɛ̌, íŋgáŋâɽi...
20       án âjt̪ɔ́ ŋíðə̀nɔ́ŋ ŋʊ̀rɔ́t̪ɛ̌, íŋgáŋâɽi...
                               ...                        
28719    kúkù kâŋtɔ́ kɔ́rìŋú nɛ̀ lɛ̀vɛ̀r dɛ̀, àn ...
28720    kúkù kâŋtɔ́ kɔ́rìŋú nɛ̀ lɛ̀vɛ̀r dɛ̀, àn ...
28724                           jɛ́ûrmùðɔ̀, jɛ́ûrmùðɔ̀
28739                       ùrɔ̀...ùrnɔ̀ kàrɔ́ t̪àŋòŋ
28755    ùrnɔ̀ kàrɔ́ t̪á àprí jàŋó...àprí jɛ́u...
Name: unicode_norm, Length: 1273, dtype: object

In [73]:
df_tone['no_punct'] = df_tone['unicode_norm'].apply(remove_punct)
df_tone.head()

,start,end,transcription,eaf_source,wav_source,unicode_norm,no_punct
0,184240,189330,mə̀nà tʃàt̪á án âjt̪ɔ́ ŋìðə̀nɔ́ŋ ŋírɔ́t...,/Users/markjos/Library/CloudStorage/GoogleDriv...,/G:/Shared drives/Tira/Recordings/2022-07-19-T...,mə̀nà tʃàt̪á án âjt̪ɔ́ ŋìðə̀nɔ́ŋ ŋírɔ́t...,mə̀nà tʃàt̪á án âjt̪ɔ́ ŋìðə̀nɔ́ŋ ŋírɔ́t...
7,271280,272310,íŋgáŋâɽìɲǎ,/Users/markjos/Library/CloudStorage/GoogleDriv...,/G:/Shared drives/Tira/Recordings/2022-07-19-T...,íŋgáŋâɽìɲǎ,íŋgáŋâɽìɲǎ
8,275030,276050,íŋgáŋâɽìɲǎ,/Users/markjos/Library/CloudStorage/GoogleDriv...,/G:/Shared drives/Tira/Recordings/2022-07-19-T...,íŋgáŋâɽìɲǎ,íŋgáŋâɽìɲǎ
9,304295,305015,íŋgáŋâɽìɲǎ,/Users/markjos/Library/CloudStorage/GoogleDriv...,/G:/Shared drives/Tira/Recordings/2022-07-19-T...,íŋgáŋâɽìɲǎ,íŋgáŋâɽìɲǎ
10,331510,333800,ŋìðə̀ní ŋrɔ̀t̪ɔ́ t̪ɔ́wə̀nì,/Users/markjos/Library/CloudStorage/GoogleDriv...,/G:/Shared drives/Tira/Recordings/2022-07-19-T...,ŋìðə̀ní ŋrɔ̀t̪ɔ́ t̪ɔ́wə̀nì,ŋìðə̀ní ŋrɔ̀t̪ɔ́ t̪ɔ́wə̀nì


# Detect rows with English

In [74]:
en_dict = enchant.request_dict('en_US')
en_dict

In [75]:
is_en_word = lambda w: en_dict.check(w) and (not has_tone(w)) and len(w)!=1

In [76]:
has_english = lambda s: any(is_en_word(w) for w in s.split())
en_rows = df_tone['no_punct'].apply(has_english)
en_rows.value_counts()


no_punct
False    26497
True       200
Name: count, dtype: int64

In [77]:
get_english_words = lambda s: ', '.join(word for word in s.split() if is_en_word(word))
get_english_words('yeah apri jicelo')

'yeah'

In [78]:
df_tone['en_words']=df_tone['no_punct'].apply(get_english_words)
df_tone[en_rows][['no_punct', 'en_words']].head()

,no_punct,en_words
709,you can say ŋɛ́n ŋɔ̂ɽàt̪à nɛ̀ lə̀vɛ̌r,"you, can, say"
710,yeah ŋɛ́n ŋɔ̂ɽàt̪à nɛ̀ lə̀vɛ̌r,yeah
722,so you can say ŋɛ́n ŋɔ́ɽít̪ɔ̀ àpɾíɲà,"so, you, can, say"
915,or àvàlà àvàlà kèclò,or
1001,Canada gônór USA,"Canada, USA"


In [79]:
en_df_path = '../data/tira-asr/has_english.csv'
df_tone[en_rows].to_csv(en_df_path, index=False)

In [80]:
df_noeng = df_tone.copy()[~en_rows]
df_noeng.shape

(26497, 8)

# Detect upper case chars

In [81]:
is_tone_word = lambda s: all(c in 'HML' for c in s)
has_tone_word = lambda s: any(is_tone_word(w) for w in s.split())
remove_tone_word = lambda s: ' '.join(word for word in s.split() if not is_tone_word(word))
df_noeng['no_tone_word']=df_noeng['no_punct'].apply(remove_tone_word)
df_noeng[df_noeng['no_punct'].apply(has_tone_word)]

,start,end,transcription,eaf_source,wav_source,unicode_norm,no_punct,en_words,no_tone_word
748,154895,156955,kə̀və̀lɛ̀ðátɛ́ únɛ̀ɾɛ̀ LLHH HLHL,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Users/ninahk/Library/CloudStorage/GoogleDrive...,kə̀və̀lɛ̀ðátɛ́ únɛ̀ɾɛ̀ LLHH HLHL,kə̀və̀lɛ̀ðátɛ́ únɛ̀ɾɛ̀ LLHH HLHL,,kə̀və̀lɛ̀ðátɛ́ únɛ̀ɾɛ̀
750,174570,176780,jɪ́və̀lɛ̀ðátɛ́ únɛ̀ɾɛ̀ HLHH HLHL,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Users/ninahk/Library/CloudStorage/GoogleDrive...,jɪ́və̀lɛ̀ðátɛ́ únɛ̀ɾɛ̀ HLHH HLHL,jɪ́və̀lɛ̀ðátɛ́ únɛ̀ɾɛ̀ HLHH HLHL,,jɪ́və̀lɛ̀ðátɛ́ únɛ̀ɾɛ̀
777,678605,681168,īŋgáŋávə̀lɛ̀ðà ǹdàbàgɛ̀ H-HL-L L-L-L L-L,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Users/ninahk/Library/CloudStorage/GoogleDrive...,īŋgáŋávə̀lɛ̀ðà ǹdàbàgɛ̀ H-HL-L L-L-L L-L,īŋgáŋávə̀lɛ̀ðà ǹdàbàgɛ̀ H HL L L L L L L,,īŋgáŋávə̀lɛ̀ðà ǹdàbàgɛ̀
2596,726520,732460,lǎllǔvə́lɛ̂ːðǎ ... ndɔ̀bàɡɛ̌ ... LH LH H H...,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Volumes/GoogleDrive/Shared drives/Tira/Record...,lǎllǔvə́lɛ̂ːðǎ ... ndɔ̀bàɡɛ̌ ... LH LH H H...,lǎllǔvə́lɛ̂ːðǎ ndɔ̀bàɡɛ̌ LH LH H HL LLL,,lǎllǔvə́lɛ̂ːðǎ ndɔ̀bàɡɛ̌
19006,771225,774634,ùrnàŋgéi kə̀və̀lɛ̀ðɔ́ ðáŋàlà LLH LLH HLL,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Volumes/GoogleDrive/Shared drives/Tira/Record...,ùrnàŋgéi kə̀və̀lɛ̀ðɔ́ ðáŋàlà LLH LLH HLL,ùrnàŋgéi kə̀və̀lɛ̀ðɔ́ ðáŋàlà LLH LLH HLL,,ùrnàŋgéi kə̀və̀lɛ̀ðɔ́ ðáŋàlà
19016,975220,978020,jɛ́vəlɛ̀ðáŋà ùnɛ̀rɛ̀ HLLHL HLH,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Volumes/GoogleDrive/Shared drives/Tira/Record...,jɛ́vəlɛ̀ðáŋà ùnɛ̀rɛ̀ HLLHL HLH,jɛ́vəlɛ̀ðáŋà ùnɛ̀rɛ̀ HLLHL HLH,,jɛ́vəlɛ̀ðáŋà ùnɛ̀rɛ̀
19021,1035723,1038078,kə̀və̀lɛ̀ðɛ́ŋí únɛ̀rɛ̀ LLLHH HLHL,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Volumes/GoogleDrive/Shared drives/Tira/Record...,kə̀və̀lɛ̀ðɛ́ŋí únɛ̀rɛ̀ LLLHH HLHL,kə̀və̀lɛ̀ðɛ́ŋí únɛ̀rɛ̀ LLLHH HLHL,,kə̀və̀lɛ̀ðɛ́ŋí únɛ̀rɛ̀
19032,1172967,1175594,kə̀və̀lɛ̀ðáŋá ûnɛ̀rɛ̀ LLLHH HLHL,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Volumes/GoogleDrive/Shared drives/Tira/Record...,kə̀və̀lɛ̀ðáŋá ûnɛ̀rɛ̀ LLLHH HLHL,kə̀və̀lɛ̀ðáŋá ûnɛ̀rɛ̀ LLLHH HLHL,,kə̀və̀lɛ̀ðáŋá ûnɛ̀rɛ̀
19045,1456032,1458595,ɲàvə̀lɛ̀ðáŋà ùnɛ̀rɛ̀ LLLHL LHL,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Volumes/GoogleDrive/Shared drives/Tira/Record...,ɲàvə̀lɛ̀ðáŋà ùnɛ̀rɛ̀ LLLHL LHL,ɲàvə̀lɛ̀ðáŋà ùnɛ̀rɛ̀ LLLHL LHL,,ɲàvə̀lɛ̀ðáŋà ùnɛ̀rɛ̀
19052,1680477,1682643,kə̀və̀lɛ̀ðálír ùnɛ̀rɛ̀ LLLHH HLHL,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Volumes/GoogleDrive/Shared drives/Tira/Record...,kə̀və̀lɛ̀ðálír ùnɛ̀rɛ̀ LLLHH HLHL,kə̀və̀lɛ̀ðálír ùnɛ̀rɛ̀ LLLHH HLHL,,kə̀və̀lɛ̀ðálír ùnɛ̀rɛ̀


In [82]:
has_upper = lambda s: s.lower()!=s
df_noeng[df_noeng['no_tone_word'].apply(has_upper)]

,start,end,transcription,eaf_source,wav_source,unicode_norm,no_punct,en_words,no_tone_word
4634,350380,352030,lɜ̀ù lɛjja ed̪ɔɽɔlɛDEL,/Users/markjos/Library/CloudStorage/GoogleDriv...,/G:/Shared drives/Tira/Recordings/2021-08-04-T...,lɜ̀ù lɛjja ed̪ɔɽɔlɛDEL,lɜ̀ù lɛjja ed̪ɔɽɔlɛDEL,,lɜ̀ù lɛjja ed̪ɔɽɔlɛDEL
18271,1464502,1465190,Ídɜ́ɽí,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Volumes/GoogleDrive/Shared drives/Tira/Record...,Ídɜ́ɽí,Ídɜ́ɽí,,Ídɜ́ɽí
18272,1476594,1478117,Ídɜ́ɽí kɪ̀cə̀ló,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Volumes/GoogleDrive/Shared drives/Tira/Record...,Ídɜ́ɽí kɪ̀cə̀ló,Ídɜ́ɽí kɪ̀cə̀ló,,Ídɜ́ɽí kɪ̀cə̀ló
18273,1480070,1481374,Ídɜ́ɽí kɪ̀cə̀ló,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Volumes/GoogleDrive/Shared drives/Tira/Record...,Ídɜ́ɽí kɪ̀cə̀ló,Ídɜ́ɽí kɪ̀cə̀ló,,Ídɜ́ɽí kɪ̀cə̀ló
18274,1484275,1485621,Ídɜ́ɽí kɪ̀cə̀ló,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Volumes/GoogleDrive/Shared drives/Tira/Record...,Ídɜ́ɽí kɪ̀cə̀ló,Ídɜ́ɽí kɪ̀cə̀ló,,Ídɜ́ɽí kɪ̀cə̀ló
18275,1490844,1492101,Ídɜ́ɽí kɪ̀cə̀ló,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Volumes/GoogleDrive/Shared drives/Tira/Record...,Ídɜ́ɽí kɪ̀cə̀ló,Ídɜ́ɽí kɪ̀cə̀ló,,Ídɜ́ɽí kɪ̀cə̀ló
18464,1464502,1465190,Ídɜ́ɽí,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Volumes/GoogleDrive/Shared drives/Tira/Record...,Ídɜ́ɽí,Ídɜ́ɽí,,Ídɜ́ɽí
18465,1476594,1478117,Ídɜ́ɽí kɪ̀cə̀ló,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Volumes/GoogleDrive/Shared drives/Tira/Record...,Ídɜ́ɽí kɪ̀cə̀ló,Ídɜ́ɽí kɪ̀cə̀ló,,Ídɜ́ɽí kɪ̀cə̀ló
18466,1480070,1481374,Ídɜ́ɽí kɪ̀cə̀ló,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Volumes/GoogleDrive/Shared drives/Tira/Record...,Ídɜ́ɽí kɪ̀cə̀ló,Ídɜ́ɽí kɪ̀cə̀ló,,Ídɜ́ɽí kɪ̀cə̀ló
18467,1484275,1485621,Ídɜ́ɽí kɪ̀cə̀ló,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Volumes/GoogleDrive/Shared drives/Tira/Record...,Ídɜ́ɽí kɪ̀cə̀ló,Ídɜ́ɽí kɪ̀cə̀ló,,Ídɜ́ɽí kɪ̀cə̀ló


In [83]:
df_noeng['lower']=df_noeng['no_tone_word'].apply(str.lower)
df_noeng.head()

,start,end,transcription,eaf_source,wav_source,unicode_norm,no_punct,en_words,no_tone_word,lower
0,184240,189330,mə̀nà tʃàt̪á án âjt̪ɔ́ ŋìðə̀nɔ́ŋ ŋírɔ́t...,/Users/markjos/Library/CloudStorage/GoogleDriv...,/G:/Shared drives/Tira/Recordings/2022-07-19-T...,mə̀nà tʃàt̪á án âjt̪ɔ́ ŋìðə̀nɔ́ŋ ŋírɔ́t...,mə̀nà tʃàt̪á án âjt̪ɔ́ ŋìðə̀nɔ́ŋ ŋírɔ́t...,,mə̀nà tʃàt̪á án âjt̪ɔ́ ŋìðə̀nɔ́ŋ ŋírɔ́t...,mə̀nà tʃàt̪á án âjt̪ɔ́ ŋìðə̀nɔ́ŋ ŋírɔ́t...
7,271280,272310,íŋgáŋâɽìɲǎ,/Users/markjos/Library/CloudStorage/GoogleDriv...,/G:/Shared drives/Tira/Recordings/2022-07-19-T...,íŋgáŋâɽìɲǎ,íŋgáŋâɽìɲǎ,,íŋgáŋâɽìɲǎ,íŋgáŋâɽìɲǎ
8,275030,276050,íŋgáŋâɽìɲǎ,/Users/markjos/Library/CloudStorage/GoogleDriv...,/G:/Shared drives/Tira/Recordings/2022-07-19-T...,íŋgáŋâɽìɲǎ,íŋgáŋâɽìɲǎ,,íŋgáŋâɽìɲǎ,íŋgáŋâɽìɲǎ
9,304295,305015,íŋgáŋâɽìɲǎ,/Users/markjos/Library/CloudStorage/GoogleDriv...,/G:/Shared drives/Tira/Recordings/2022-07-19-T...,íŋgáŋâɽìɲǎ,íŋgáŋâɽìɲǎ,,íŋgáŋâɽìɲǎ,íŋgáŋâɽìɲǎ
10,331510,333800,ŋìðə̀ní ŋrɔ̀t̪ɔ́ t̪ɔ́wə̀nì,/Users/markjos/Library/CloudStorage/GoogleDriv...,/G:/Shared drives/Tira/Recordings/2022-07-19-T...,ŋìðə̀ní ŋrɔ̀t̪ɔ́ t̪ɔ́wə̀nì,ŋìðə̀ní ŋrɔ̀t̪ɔ́ t̪ɔ́wə̀nì,,ŋìðə̀ní ŋrɔ̀t̪ɔ́ t̪ɔ́wə̀nì,ŋìðə̀ní ŋrɔ̀t̪ɔ́ t̪ɔ́wə̀nì


# Character normalization

In [84]:
# look at all unique chars, then make replacements as deemed fit
unique_chars = set()
df_noeng['lower'].apply(unique_chars.update)
unique_chars, len(unique_chars)

({' ',
  'a',
  'b',
  'c',
  'd',
  'e',
  'f',
  'g',
  'h',
  'i',
  'j',
  'k',
  'l',
  'm',
  'n',
  'o',
  'p',
  'r',
  's',
  't',
  'u',
  'v',
  'w',
  'z',
  'ð',
  'ŋ',
  'œ',
  'ɐ',
  'ɑ',
  'ɔ',
  'ɕ',
  'ɖ',
  'ɘ',
  'ə',
  'ɛ',
  'ɜ',
  'ɞ',
  'ɟ',
  'ɡ',
  'ɣ',
  'ɦ',
  'ɨ',
  'ɪ',
  'ɭ',
  'ɯ',
  'ɰ',
  'ɲ',
  'ɳ',
  'ɹ',
  'ɺ',
  'ɻ',
  'ɽ',
  'ɾ',
  'ʃ',
  'ʈ',
  'ʊ',
  'ʑ',
  'ʒ',
  'ʔ',
  'ʝ',
  'ː',
  '̀',
  '́',
  '̂',
  '̃',
  '̄',
  '̆',
  '̊',
  '̋',
  '̌',
  '̏',
  '̘',
  '̞',
  '̥',
  '̧',
  '̩',
  '̪',
  '̯',
  '̰',
  '̺',
  '͡',
  'β',
  'θ',
  '᷉',
  '–',
  'ꜛ',
  'ꜜ'},
 87)

In [85]:
reps = {
  # ' ',
  # 'a',
  # 'b',
  # 'c',
  # 'd',
  # 'e',
  # 'f',
  # 'g',
  # 'h',
  # 'i',
  # 'j',
  # 'k',
  # 'l',
  # 'm',
  # 'n',
  # 'o',
  # 'p',
  # 'r',
  # 's',
  # 't',
  # 'u',
  # 'v',
  # 'w',
  'z': 'ð', # typo
  # 'ð',
  # 'ŋ',
  'œ': 'ɔ', # fronted allophone of /ɔ/
  'ɐ': 'ɜ', # alternate spelling of [ɜ]
  'ɑ': 'a', # alternate spelling of [a]
  # 'ɔ',
  'tɕ': 'c', # normalize tɕ to c
  'ɖ': 'd', # normalize retro to alv
  'ɘ': 'ə', # alternate spelling of [ə]
  # 'ə',
  # 'ɛ',
  # 'ɜ',
  'ɞ': 'ɔ', # centralized variant of /ɔ/
  # 'ɟ',
  'ɡ': 'g',
  'ɣ': 'g', # allophone of /g/
  'ɦ': 'h', # va
  'ɨ': 'i', # allophone of /i/
  'ɪ': 'i', # allophone of /i/
  # 'ɭ', 
  'ɯ': 'u', # allophone of /u/ (or typo?)
  'ɰ': 'g',
  # 'ɲ',
  'ɳ': 'n', # normalize retro to alveolar
  'ɹ': 'r', # normalize rhotics
  'ɺ': 'ɽ', # normalize rhotics
  'ɻ': 'ɽ', # normalize rhotics
  # 'ɽ',
  # 'ɾ',
  'tʃ': 'c', # normalize tʃ to c
  'ʈ': 't', # normalize retro to alveolar
  # 'ʊ',
  'dʑ': 'ɟ', # normalize dʑ to ɟ
  'dʒ': 'ɟ', # normalize dʒ to ɟ
  'ʔ': '', # glottal stop non-phonemic
  'ʝ': 'ɟ', # ʝ allophone of ɟ
  'ː': '', # length non-phonemic
  # '̀',
  # '́',
  # '̂',
  '̃': '', # nasality non-phonemic
  # '̄',
  '̆': '\u030C', # normalize breve to caron for LH
  '̊': '', # not sure how this got here
  '̋': '\u0301', # normalize super-high to high
  # '̌',
  '̏': '\u0300', # normalize super-low to low
  '̘': '', # ignore advanced diacritic
  '̞': '', # ignore lowering diacritic
  '̥': '', # ignore voiceless diacritic
  '̧': '', # ignore cedilla tail
  '̩': '', # ignore syllabic diacritic
  # '̪',
  '̯': '', # ignore non-syllabic diacritic
  '̰': '', # ignore creaky diacritic
  '̺': '̪', # replace laminal diacritic with dental
  '͡': '', # ignore tie bar 
  'β': 'b', # allophone of /b/
  'θ': 'ð', # allophone of /ð/
  '᷉': '\u0302', # normalize falling-rising to falling
  '–': '',
  'ꜛ': '', # ignore upstep
  'ꜜ': '', # ignore downstep
  'ʒ': 'ʒ',
  # 'ʃ': 'ʃ', # ʃ appears in dataset when name "Sharon" is used in Tira sentence
  'ʑ': 'ʑ',
  'ɕ': 'ɕ',
}

In [86]:
def make_reps(s):
    for k, v in reps.items():
        s = s.replace(k, v)
    return s
contains_rep_char = lambda s: any(c in reps.keys() for c in s)
df_noeng['rep_norm'] = df_noeng['lower']
i=0
while True in df_noeng['rep_norm'].apply(contains_rep_char).value_counts():
    i+=1
    print("Iteration", i)
    df_noeng['rep_norm']=df_noeng['rep_norm'].apply(make_reps)
df_noeng.head()

Iteration 1
Iteration 2


,start,end,transcription,eaf_source,wav_source,unicode_norm,no_punct,en_words,no_tone_word,lower,rep_norm
0,184240,189330,mə̀nà tʃàt̪á án âjt̪ɔ́ ŋìðə̀nɔ́ŋ ŋírɔ́t...,/Users/markjos/Library/CloudStorage/GoogleDriv...,/G:/Shared drives/Tira/Recordings/2022-07-19-T...,mə̀nà tʃàt̪á án âjt̪ɔ́ ŋìðə̀nɔ́ŋ ŋírɔ́t...,mə̀nà tʃàt̪á án âjt̪ɔ́ ŋìðə̀nɔ́ŋ ŋírɔ́t...,,mə̀nà tʃàt̪á án âjt̪ɔ́ ŋìðə̀nɔ́ŋ ŋírɔ́t...,mə̀nà tʃàt̪á án âjt̪ɔ́ ŋìðə̀nɔ́ŋ ŋírɔ́t...,mə̀nà càt̪á án âjt̪ɔ́ ŋìðə̀nɔ́ŋ ŋírɔ́t̪...
7,271280,272310,íŋgáŋâɽìɲǎ,/Users/markjos/Library/CloudStorage/GoogleDriv...,/G:/Shared drives/Tira/Recordings/2022-07-19-T...,íŋgáŋâɽìɲǎ,íŋgáŋâɽìɲǎ,,íŋgáŋâɽìɲǎ,íŋgáŋâɽìɲǎ,íŋgáŋâɽìɲǎ
8,275030,276050,íŋgáŋâɽìɲǎ,/Users/markjos/Library/CloudStorage/GoogleDriv...,/G:/Shared drives/Tira/Recordings/2022-07-19-T...,íŋgáŋâɽìɲǎ,íŋgáŋâɽìɲǎ,,íŋgáŋâɽìɲǎ,íŋgáŋâɽìɲǎ,íŋgáŋâɽìɲǎ
9,304295,305015,íŋgáŋâɽìɲǎ,/Users/markjos/Library/CloudStorage/GoogleDriv...,/G:/Shared drives/Tira/Recordings/2022-07-19-T...,íŋgáŋâɽìɲǎ,íŋgáŋâɽìɲǎ,,íŋgáŋâɽìɲǎ,íŋgáŋâɽìɲǎ,íŋgáŋâɽìɲǎ
10,331510,333800,ŋìðə̀ní ŋrɔ̀t̪ɔ́ t̪ɔ́wə̀nì,/Users/markjos/Library/CloudStorage/GoogleDriv...,/G:/Shared drives/Tira/Recordings/2022-07-19-T...,ŋìðə̀ní ŋrɔ̀t̪ɔ́ t̪ɔ́wə̀nì,ŋìðə̀ní ŋrɔ̀t̪ɔ́ t̪ɔ́wə̀nì,,ŋìðə̀ní ŋrɔ̀t̪ɔ́ t̪ɔ́wə̀nì,ŋìðə̀ní ŋrɔ̀t̪ɔ́ t̪ɔ́wə̀nì,ŋìðə̀ní ŋrɔ̀t̪ɔ́ t̪ɔ́wə̀nì


# Drop unneeded columns

In [87]:
df_out = df_noeng.copy()
df_out['raw_transcription']=df_out['transcription']
df_out['transcription']=df_out['rep_norm']
dropcols = [
    'unicode_norm', 'no_punct', 'en_words', 'no_tone_word', 'lower', 'rep_norm',
]
df_out = df_out.drop(dropcols, axis=1)
df_out.columns

Index(['start', 'end', 'transcription', 'eaf_source', 'wav_source',
       'raw_transcription'],
      dtype='object')

In [88]:
df_out.to_csv('../data/tira-asr/metadata.csv',index=False)

# Fix broken wav paths

In [5]:
clipdata_path = '../data/tira-asr/clipdata.csv'
clip_df = pd.read_csv(clipdata_path, keep_default_na=False)
clip_df.head()

,start,end,transcription,eaf_source,wav_source,raw_transcription,clip
0,184240,189330,mə̀nà càt̪á án âjt̪ɔ́ ŋìðə̀nɔ́ŋ ŋírɔ́t̪...,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Users/markjos/Library/CloudStorage/GoogleDriv...,mə̀nà tʃàt̪á án âjt̪ɔ́ ŋìðə̀nɔ́ŋ ŋírɔ́t...,data/tira-asr/clips/HH20220719-1-m03s04ms240-m...
1,271280,272310,íŋgáŋâɽìɲǎ,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Users/markjos/Library/CloudStorage/GoogleDriv...,íŋgáŋâɽìɲǎ,data/tira-asr/clips/HH20220719-1-m04s31ms280-m...
2,275030,276050,íŋgáŋâɽìɲǎ,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Users/markjos/Library/CloudStorage/GoogleDriv...,íŋgáŋâɽìɲǎ,data/tira-asr/clips/HH20220719-1-m04s35ms030-m...
3,304295,305015,íŋgáŋâɽìɲǎ,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Users/markjos/Library/CloudStorage/GoogleDriv...,íŋgáŋâɽìɲǎ,data/tira-asr/clips/HH20220719-1-m05s04ms295-m...
4,331510,333800,ŋìðə̀ní ŋrɔ̀t̪ɔ́ t̪ɔ́wə̀nì,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Users/markjos/Library/CloudStorage/GoogleDriv...,ŋìðə̀ní ŋrɔ̀t̪ɔ́ t̪ɔ́wə̀nì,data/tira-asr/clips/HH20220719-1-m05s31ms510-m...


In [10]:
clip_counts=clip_df['clip'].value_counts()
clip_counts[clip_counts==2]

clip
data/tira-asr/clips/HH20230731-2-m17s18ms110-m17s21ms620.wav    2
data/tira-asr/clips/HH20230724-m41s57ms130-m41s59ms540.wav      2
data/tira-asr/clips/HH20230724-m41s28ms210-m41s29ms470.wav      2
data/tira-asr/clips/HH20230724-m41s18ms940-m41s20ms170.wav      2
data/tira-asr/clips/HH20230724-m40s55ms400-m40s57ms010.wav      2
                                                               ..
data/tira-asr/clips/HH20220201-m10s58ms270-m10s59ms510.wav      2
data/tira-asr/clips/HH20220201-m10s53ms900-m10s55ms130.wav      2
data/tira-asr/clips/HH20220201-m12s53ms370-m12s54ms850.wav      2
data/tira-asr/clips/HH20220201-m12s43ms780-m12s45ms410.wav      2
data/tira-asr/clips/HH20220201-m10s36ms570-m10s38ms120.wav      2
Name: count, Length: 1087, dtype: int64